## Load model

### Input parameters

In [164]:
dataset_file = './trainingdata_stepwise_turkish.tsv'
model_load_file = './saved_models/gestnet_8_200.pt'

### Load the packages and functions

In [165]:
from dev import *
import re
import pandas as pd
import numpy as np
import statsmodels.api as sm

data_stepwise = Dataset(dataset_file)
model = Seq2Seq(training_data=data_stepwise)

/Users/hillel/opt/anaconda3/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/Users/hillel/opt/anaconda3/lib/python3.8/site-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/Users/hillel/opt/anaconda3/lib/python3.8/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most r

#### Create a new model or load a previous one

In [166]:
# load a previous model
if model_load_file != 'none':
    try:
        model = Seq2Seq(load=model_load_file)
    except:
        model.train_model(training_data=data_stepwise, n_epochs=200)
# create a new model
if model_load_file == 'none':
    model.train_model(training_data=data_stepwise, n_epochs=200)

  6%|▋         | 13/200 [00:16<03:54,  1.26s/it]


KeyboardInterrupt: 

## Evaluate model

### Run the model on all 2 syllable words

In [132]:
# load dataframe
data = pd.read_csv(dataset_file, sep='\t')
data = data[data['syllables']==2]

# helper function to get decoder outputs
def get_decoder(input: torch.Tensor, target: torch.Tensor) -> np.ndarray:
    with torch.no_grad():
        _, attn_map_seq = model(input, target)
    return attn_map_seq.numpy()[:,0] # attention paid to the first letter

# helper functions to get correct inputs
def get_trial(training_data, word):
    trial = training_data.make_trial(word)
    return trial[0], torch.cat((trial[1], trial[2]), axis=1)

# get the decoder outputs for each word
get_out = lambda x, y : pd.DataFrame(get_decoder(*get_trial(x, y)))
df = get_out(data_stepwise, data['underlying'].values[0]).T

for i in range(1, data['underlying'].shape[0]):
    df = pd.concat(
        (df, get_out(data_stepwise, data['underlying'].values[i]).T),
        axis=0
    )

df = df.reset_index().drop('index', axis=1)

# add columns
for c in ['underlying', 'consonant', 'vowel']:
    col = data[c]
    col = col.reset_index().drop('index', axis=1)
    df[c] = col

df = df.rename({'vowel': "V2"}, axis=1)
df = df.assign(
    V1 = lambda d: d['underlying'].astype(str).str[0]
)

print(df)

           0         1         2         3         4         5         6  \
0   0.842151  0.685471  0.676226  0.525379  0.021649  0.384931  0.033987   
1   0.897407  0.956952  0.949624  0.901668  0.419069  0.798932  0.198599   
2   0.852523  0.772747  0.761797  0.633111  0.073592  0.529568  0.082031   
3   0.621409  0.543160  0.592652  0.618745  0.402602  0.404765  0.177946   
4   0.657484  0.459738  0.548320  0.601951  0.266484  0.297078  0.109275   
5   0.610877  0.418875  0.506909  0.528937  0.186519  0.240244  0.083049   
6   0.682967  0.590527  0.649539  0.657843  0.316262  0.335354  0.111387   
7   0.942226  0.835873  0.771145  0.697956  0.033406  0.517991  0.066218   
8   0.853852  0.645232  0.630860  0.486186  0.021859  0.425644  0.025895   
9   0.929844  0.942736  0.942647  0.906243  0.317740  0.673207  0.051936   
10  0.871312  0.744662  0.733025  0.614336  0.070109  0.542739  0.056347   
11  0.620709  0.525611  0.577177  0.615275  0.345125  0.322897  0.036263   
12  0.654035

### Prep dataframe for analysis

In [133]:
# create additional categorical values
df_a = df.assign(
    rounded = lambda d: d["V1"].apply(lambda y: 1 if y in ["ø", "u", "y", "o"] else 0)
)
df_b = df_a.assign(
    fronted = lambda d: d["V1"].apply(lambda y: 1 if y in ["ø", "e", "y", "i"] else 0)
)
df_c = df_b.assign(
    high = lambda d: d["V1"].apply(lambda y: 1 if y in["ø", "u", "y", "o", "i", "ɯ"] else 0)
)

In [134]:
df_melt = pd.melt(
    frame=df_c,
    id_vars=["V1", "V2", "consonant", "underlying", "fronted", "rounded", "high"],
    value_name="Attention",
    value_vars=[5, 6, 7, 8, 9],
    var_name="Time"
)

# set the categories as well
df_mle = df_melt.astype(
    {
        "Time": 'int64', 
        "V1": 'category', 
        "V2": 'category', 
        "consonant": 'category', 
        "fronted": 'category', 
        "rounded": 'category', 
        "high": 'category', 
        "underlying": 'category'
    }
)
print(df_mle)

    V1 V2 consonant underlying fronted rounded high  Time  Attention
0    i  H         b       ib-H       1       0    1     5   0.384931
1    a  H         b       ab-H       0       0    0     5   0.798932
2    e  H         b       eb-H       1       0    0     5   0.529568
3    o  H         b       ob-H       0       1    1     5   0.404765
4    u  H         b       ub-H       0       1    1     5   0.297078
..  .. ..       ...        ...     ...     ...  ...   ...        ...
235  o  L         d       od-L       0       1    1     9   0.618236
236  u  L         d       ud-L       0       1    1     9   0.578829
237  y  L         d       yd-L       1       1    1     9   0.652646
238  ø  L         d       ød-L       1       1    1     9   0.670048
239  ɯ  L         d       ɯd-L       0       0    1     9   0.927472

[240 rows x 9 columns]


### Run the analysis on the data

In [135]:
# data = sm.datasets.get_rdataset("dietox", "geepack").data
# print(data.dtypes)

In [163]:
import statsmodels.formula.api as smf

''' From paper:
We used the identity of input V2 as 
    - either a high harmony trigger (/i, u/) 
      or a non-high non-trigger
    - and decoder timepoint as main factors, 
    - model as a random factor, 
    - and the attention value assigned to the encoder hidden state 
        associated with input V2 as the dependent variable.

This result suggests that the
decoder learns to pay more attention to a V2 at an
earlier timepoint when that V2 is a harmony trigger,
consistent with the representation of an anticipatory
(early-activating) gesture assumed by the Gestural
Harmony Model.
'''

md = smf.ols("Attention ~ Time + V2 + fronted * high * rounded", df_mle, groups=df_mle["underlying"])
mdf = md.fit()
print(mdf.summary())

         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: Attention
No. Observations: 240     Method:             REML     
No. Groups:       48      Scale:              0.0329   
Min. group size:  5       Log-Likelihood:     60.4063  
Max. group size:  5       Converged:          Yes      
Mean group size:  5.0                                  
-------------------------------------------------------
            Coef.  Std.Err.    z    P>|z| [0.025 0.975]
-------------------------------------------------------
Intercept   -0.611    0.060 -10.139 0.000 -0.729 -0.493
V2[T.L]     -0.089    0.023  -3.814 0.000 -0.135 -0.043
Time         0.147    0.008  17.744 0.000  0.131  0.163
Group Var    0.000    0.013                            



/Users/hillel/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/mixed_linear_model.py:1634: UserWarning: Random effects covariance is singular
  warnings.warn(msg)
/Users/hillel/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [159]:
# get means of V2 groups
df_mle.groupby(['V2', 'Time'], as_index=False).agg({'Attention':'mean'})

,V2,Time,Attention
0,H,5,0.306102
1,H,6,0.217068
2,H,7,0.252178
3,H,8,0.551307
4,H,9,0.759610
5,L,5,0.167382
6,L,6,0.056469
7,L,7,0.104369
8,L,8,0.534676
9,L,9,0.776808


In [152]:
from statsmodels.stats.multicomp import MultiComparison

mc = MultiComparison(df_mle['Attention'], groups=df_mle['V2'])
mc.tukeyhsd().summary()

group1,group2,meandiff,p-adj,lower,upper,reject
H,L,-0.0893,0.0129,-0.1596,-0.0191,True
